# Atelier CQL avancé — Fleet Tracing (IoT) — Formateur (corrigé)

Ce notebook contient:
- le schéma complet
- un seed minimal
- les requêtes solutions
- un bloc pitfalls + corrections


## 00 — Schéma (solution)
```sql
CREATE KEYSPACE IF NOT EXISTS atelier
WITH replication = {'class':'SimpleStrategy','replication_factor':1};

USE atelier;

CREATE TABLE IF NOT EXISTS devices_by_fleet (
  fleet_id text,
  device_id text,
  model text,
  activated_at timestamp,
  PRIMARY KEY ((fleet_id), device_id)
);

CREATE TABLE IF NOT EXISTS latest_telemetry_by_device (
  device_id text PRIMARY KEY,
  last_ts timestamp,
  lat double,
  lon double,
  speed_kmh double,
  battery_pct int,
  temp_c double
);

CREATE TABLE IF NOT EXISTS telemetry_by_device_day (
  device_id text,
  day date,
  ts timestamp,
  lat double,
  lon double,
  speed_kmh double,
  battery_pct int,
  temp_c double,
  zone text,
  PRIMARY KEY ((device_id, day), ts)
) WITH CLUSTERING ORDER BY (ts DESC);

CREATE TABLE IF NOT EXISTS alerts_by_fleet_day (
  fleet_id text,
  day date,
  severity text,
  ts timestamp,
  device_id text,
  alert_type text,
  message text,
  PRIMARY KEY ((fleet_id, day, severity), ts, device_id)
) WITH CLUSTERING ORDER BY (ts DESC);

CREATE TABLE IF NOT EXISTS telemetry_by_fleet_day (
  fleet_id text,
  day date,
  ts timestamp,
  device_id text,
  lat double,
  lon double,
  speed_kmh double,
  battery_pct int,
  temp_c double,
  PRIMARY KEY ((fleet_id, day), ts, device_id)
) WITH CLUSTERING ORDER BY (ts DESC);
```


## 01 — Seed minimal (solution)
```sql
USE atelier;

INSERT INTO devices_by_fleet (fleet_id, device_id, model, activated_at)
VALUES ('FLEET_PARIS','BUS-001','GPS-TX-1','2025-12-01 08:00:00+0000');
INSERT INTO devices_by_fleet (fleet_id, device_id, model, activated_at)
VALUES ('FLEET_PARIS','BUS-002','GPS-TX-1','2025-12-01 08:05:00+0000');
INSERT INTO devices_by_fleet (fleet_id, device_id, model, activated_at)
VALUES ('FLEET_LYON','TRUCK-017','GPS-TX-2','2025-12-05 09:00:00+0000');

INSERT INTO telemetry_by_device_day (device_id, day, ts, lat, lon, speed_kmh, battery_pct, temp_c, zone)
VALUES ('BUS-001','2025-12-17','2025-12-17 20:00:00+0000',48.8566,2.3522,32.5,88,36.2,'PARIS-A');
INSERT INTO telemetry_by_device_day (device_id, day, ts, lat, lon, speed_kmh, battery_pct, temp_c, zone)
VALUES ('BUS-001','2025-12-17','2025-12-17 20:05:00+0000',48.8570,2.3530,35.2,87,36.4,'PARIS-A');
INSERT INTO telemetry_by_device_day (device_id, day, ts, lat, lon, speed_kmh, battery_pct, temp_c, zone)
VALUES ('BUS-001','2025-12-17','2025-12-17 20:10:00+0000',48.8580,2.3542,10.1,86,41.8,'PARIS-A');

INSERT INTO latest_telemetry_by_device (device_id, last_ts, lat, lon, speed_kmh, battery_pct, temp_c)
VALUES ('BUS-001','2025-12-17 20:10:00+0000',48.8580,2.3542,10.1,86,41.8);

INSERT INTO alerts_by_fleet_day (fleet_id, day, severity, ts, device_id, alert_type, message)
VALUES ('FLEET_PARIS','2025-12-17','HIGH','2025-12-17 20:10:00+0000','BUS-001','TEMP_HIGH','Température > 40C');
```


## 02 — Requêtes (solutions)
```sql
USE atelier;

SELECT * FROM devices_by_fleet WHERE fleet_id='FLEET_PARIS';

SELECT * FROM latest_telemetry_by_device WHERE device_id='BUS-001';

SELECT ts, speed_kmh, battery_pct, temp_c, zone
FROM telemetry_by_device_day
WHERE device_id='BUS-001' AND day='2025-12-17'
LIMIT 5;

SELECT ts, speed_kmh, battery_pct, temp_c
FROM telemetry_by_device_day
WHERE device_id='BUS-001' AND day='2025-12-17'
AND ts >= '2025-12-17 20:00:00+0000'
AND ts <= '2025-12-17 20:06:00+0000';

SELECT ts, device_id, alert_type, message
FROM alerts_by_fleet_day
WHERE fleet_id='FLEET_PARIS' AND day='2025-12-17' AND severity='HIGH'
LIMIT 50;
```


## 03 — Pitfalls + corrections (B)
### Pitfall: `ALLOW FILTERING`
- Cassandra n'est pas conçu pour scanner et filtrer.
- Une requête qui ne cible pas la partition key implique des lectures massives et non prévisibles.

```sql
USE atelier;

CREATE TABLE IF NOT EXISTS telemetry_naive (
  device_id text,
  ts timestamp,
  fleet_id text,
  lat double,
  lon double,
  PRIMARY KEY ((device_id), ts)
) WITH CLUSTERING ORDER BY (ts DESC);

SELECT * FROM telemetry_naive WHERE fleet_id='FLEET_PARIS' ALLOW FILTERING;

-- Correction query-first + bucketing
INSERT INTO telemetry_by_fleet_day (fleet_id, day, ts, device_id, lat, lon, speed_kmh, battery_pct, temp_c)
VALUES ('FLEET_PARIS','2025-12-17','2025-12-17 20:10:00+0000','BUS-001',48.8580,2.3542,10.1,86,41.8);

SELECT ts, device_id, temp_c
FROM telemetry_by_fleet_day
WHERE fleet_id='FLEET_PARIS' AND day='2025-12-17'
LIMIT 50;

-- TTL (7 jours) + discussion tombstones
INSERT INTO telemetry_by_device_day (device_id, day, ts, lat, lon, speed_kmh, battery_pct, temp_c, zone)
VALUES ('BUS-001','2025-12-17','2025-12-17 20:15:00+0000',48.8590,2.3550,12.0,85,39.5,'PARIS-A')
USING TTL 604800;
```

### BATCH (discussion)
- Ne pas présenter comme un bulk insert magique
- Usage possible pour synchroniser un historique + un latest (à discuter / prudence)
